# Ejemplo Kafka

1. Inicializamos la SparkSession.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import os
import pyspark

spark = SparkSession.builder\
  .appName("kafka-example-1")\
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .getOrCreate()

sc =spark.sparkContext
print("Versión: ",spark.version)


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eeffcddf-ff70-42fe-bbfd-8cb876d1906b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

KeyboardInterrupt: 

2. Creamos o streaming DataFrame a partir dun fluxo de entrada de Kafka:
> Antes de crear  o dataframe é desexable crear o topic no contedor de kafka:

```bash
/opt/kafka/bin/kafka-topics.sh \
  --create \
  --topic entrada \
  --bootstrap-server kafka:9092 \
  --partitions 1 \
  --replication-factor 1

```


In [2]:
# Subscríbese a un único tópico de Kafka
df = (
    spark
        # Créase un DataFrame de streaming
        .readStream
        # Indícase que a fonte de datos é Kafka
        .format("kafka")
        # Configúrase o servidor bootstrap de Kafka
        .option("kafka.bootstrap.servers", "kafka:9092")
        # Especifícase o tópico ao que se vai subscribir
        .option("subscribe", "entrada")
        # Cárganse os datos en streaming
        .load()
)

# Selecciónase a columna value e convértese de binario a texto (STRING)
df_str = df.selectExpr("CAST(value AS STRING) as value")

# Devólvese o DataFrame resultante
df_str


DataFrame[value: string]

3. Preparamos el DataFrame de salida:

In [3]:
# Impórtase col para referenciar columnas dun DataFrame de forma explícita
from pyspark.sql.functions import col

# Sepárase o contido textual da columna value en palabras:
# - split(..., " ") converte cada liña nun array de palabras
# - explode(...) transforma ese array en múltiples filas (unha por palabra)
words = df_str.select(
    explode(
        split(col("value"), " ")
    ).alias("word")
)

# Agrúpase por palabra e calcúlase cantas veces aparece cada unha
word_counts = words.groupBy("word").count()

# Prepárase a saída co formato típico de Kafka (key/value como bytes):
# aquí convértese a STRING para poder enviar a Kafka (logo Kafka codifícao a bytes)
output = word_counts.selectExpr(
    "CAST(word AS STRING) AS key",
    "CAST(count AS STRING) AS value"
)

# Devólvese o DataFrame final listo para escribir en Kafka (key, value)
output


DataFrame[key: string, value: string]

4. Iniciamos a consulta en streaming indicando o destino dos datos:

> Antes de facelo é necesario crear o topic no contedor de kafka:

```bash
/opt/kafka/bin/kafka-topics.sh \
  --create \
  --topic salida \
  --bootstrap-server kafka:9092 \
  --partitions 1 \
  --replication-factor 1

```

In [4]:
# Configúrase a escritura do stream para enviar os resultados a Kafka
query = (
    output.writeStream
        # Indícase que o destino de saída é Kafka
        .format("kafka")
        # Configúrase o broker de Kafka ao que se conectará Spark
        .option("kafka.bootstrap.servers", "kafka:9092")
        # Especifícase o tópico de saída onde se publicarán as mensaxes
        .option("topic", "salida")
        # Indícase a localización do checkpoint para gardar o estado e o progreso do streaming
        # (imprescindible para tolerancia a fallos e para operacións con estado como groupBy/count)
        .option("checkpointLocation", "file:///home/hadoop/kafka-wordcount-checkpoint11")
        # En modo update só se envían a Kafka as filas que cambiaron en cada micro-batch
        .outputMode("update")
        # Iníciase a consulta de streaming
        .start()
)



Abrimos dúas consolas no contedor `kafka` executando este comando en dúas terminais distintas:
```bash
docker exec -it kafka bash
```

- No **terminal 1** executamos este comando:
```bash
kafka-console-producer.sh \
  --bootstrap-server kafka:9092 \
  --topic entrada
```

- No **terminal 2** executamos este comando:

```bash
kafka-console-consumer.sh \
  --bootstrap-server kafka:9092 \
  --topic salida \
  --from-beginning \
  --property print.key=true \
  --property key.separator=" => "
```

- Escribimos liñas no **terminal 1** e observamos como se vai actualizando o `wordcount`  no **terminal 2**.